In [4]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [9]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.85,
	"longitude": -87.65,
	"start_date": "2020-12-30",
	"end_date": "2024-11-30",
	"hourly": ["temperature_2m", "precipitation", "rain", "snowfall", "wind_speed_10m", "wind_speed_100m", "wind_direction_100m"],
	"timezone": "America/Chicago"
}
responses = openmeteo.weather_api(url, params=params)

In [10]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(5).ValuesAsNumpy()
hourly_wind_direction_100m = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_direction_100m"] = hourly_wind_direction_100m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)
hourly_dataframe.to_csv("weather_data.csv")

Coordinates 41.8629150390625°N -87.64877319335938°E
Elevation 179.0 m asl
Timezone b'America/Chicago' b'CST'
Timezone difference to GMT+0 -21600 s
                           date  temperature_2m  precipitation  rain  \
0     2020-12-30 06:00:00+00:00         -0.1435            1.8   1.8   
1     2020-12-30 07:00:00+00:00          0.0565            1.6   1.6   
2     2020-12-30 08:00:00+00:00          0.5065            1.2   1.2   
3     2020-12-30 09:00:00+00:00          0.8565            0.9   0.9   
4     2020-12-30 10:00:00+00:00          1.5565            1.6   1.6   
...                         ...             ...            ...   ...   
34363 2024-12-01 01:00:00+00:00         -5.3435            0.0   0.0   
34364 2024-12-01 02:00:00+00:00         -5.5435            0.0   0.0   
34365 2024-12-01 03:00:00+00:00         -5.8435            0.0   0.0   
34366 2024-12-01 04:00:00+00:00         -6.1935            0.0   0.0   
34367 2024-12-01 05:00:00+00:00         -6.6435            0.

In [ ]:
weather_df = pd.read_csv('weather_data.csv')
weather_df.drop(columns=['Unnamed: 0'], inplace=True)
# weather_df['utc_date'] = pd.to_datetime(weather_df['date'])
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.tz_convert('America/Chicago')
weather_df.to_csv('weather_data.csv', index=False)

In [3]:
import pandas as pd

weather_df = pd.read_csv('weather_data.csv')
weather_df['date'].dt.tz_localize(None)

AttributeError: Can only use .dt accessor with datetimelike values